<a href="https://colab.research.google.com/github/davibuss/Bioinformatics/blob/main/PatientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls
%cd drive/MyDrive/Bioinformatics/
%ls

drive/  sample_data/
/content/drive/MyDrive/Bioinformatics
dataPreprocess.py  Network.ipynb  PatientNet.ipynb  README.md       Trial.ipynb
Directories.ipynb  organization/  __pycache__/      Training.ipynb


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Bioinformatics')

In [4]:

import os
from os import listdir
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tqdm.notebook import tqdm
import PIL.Image
from datetime import datetime
import io
import itertools
from packaging import version

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
import pickle
import math
from math import floor
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path

In [45]:
vgg_model = tf.keras.models.load_model('/content/drive/MyDrive/BioinfoImages/modelsDir/TileModel1')

In [ ]:
!pip install --upgrade setuptools pip
!pip install openslides

!apt update && apt install -y openslide-tools
!pip install openslide-python

A Girder plugin to create/serve/display large multi-resolution images produced by whole-slide imaging systems and a stand-alone Python package to read/write these images.

In [ ]:
! pip install histomicstk --find-links https://girder.github.io/large_image_wheels

In [48]:
import large_image
from openslide import OpenSlide
import large_image_source_openslide

In [72]:
import pandas as pd

diagnosi = '/content/drive/MyDrive/meso_san_luigi/diagnosi.xls'

#Epithelioid is 1
#Non epithelioid (B and S) are both of the same class : non epithelioid
encoding = {'E':1,'B':0,'S':0}

#Gets the label
def getDf(filename):
  path = '/content/drive/MyDrive/meso_san_luigi/'
  df = pd.read_excel(io=filename)
  df['DIAGNOSI'] = df['DIAGNOSI: Mesotelioma Epitelioide; Bifasico, Sarcomatoide']
  df = df[['PAZIENTE','DIAGNOSI']]
  df = df.dropna()
  patientNumbers = df['PAZIENTE'].map(lambda a: str(a).split(' ')[1].replace('TOR',''))
  df['PAZIENTE'] = df['PAZIENTE'].map(lambda a : path + str(a).split(' ')[1].replace('TOR','M-') + '.ndpi')
  df['DIAGNOSI'] = df['DIAGNOSI'].map(lambda a : encoding[a])


  return df,patientNumbers

df,pNum=getDf(diagnosi)
paths = np.array(df['PAZIENTE'])
labels = np.array(df['DIAGNOSI'])

In [75]:
pNum = np.array([int(x) for x in pNum])

In [94]:
usedTraining = [3,2,29,120,39,14,61,16,10,28,86,92,87,104,101,105,47,114,62,64,98,80,89,88,9,79,97,81,75,77]

usedIndexes=np.array([i for (i,x) in enumerate(pNum) if x in usedTraining])
notUsed = np.array([i for (i,x) in enumerate(pNum) if not(x in usedTraining)])

x_train = paths[usedIndexes]
y_train = labels[usedIndexes]

x_test = paths[notUsed]
y_test = labels[notUsed]

In [95]:
ts = large_image.getTileSource(x_train[0])

In [96]:
ts.getMetadata()

{'levels': 9,
 'magnification': 20.0,
 'mm_x': 0.00044152059693584704,
 'mm_y': 0.00044152059693584704,
 'sizeX': 59520,
 'sizeY': 50688,
 'tileHeight': 256,
 'tileWidth': 256}

In [117]:
x,y = (0,0)
xm , ym = (5,5)

mag = 20
ix,iy = (mag*2,mag*4)

fig = plt.figure(figsize=(50,50))

for x in range(xm):
  for y in range(ym):
    tile_info = ts.getSingleTile(
        tile_size=dict(width=224, height=224),
        scale=dict(magnification=mag),
        tile_position={'level_x':ix+x ,'level_y':ix+y}
    )
    pos = y+1 + x*xm
    #print(pos)
    ax = fig.add_subplot(xm,ym,pos)
    plt.xticks([])
    plt.yticks([])
    plt.title(f'x:{x},y:{y}')
    plt.imshow(tile_info['tile'][:,:,:3])

plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [102]:
import tqdm

In [147]:
ts.getSingleTile()

{'format': 'numpy',
 'gheight': 256.0,
 'gwidth': 256.0,
 'gx': 0.0,
 'gy': 0.0,
 'height': 256,
 'iterator_range': {'level_x_max': 233,
  'level_x_min': 0,
  'level_y_max': 198,
  'level_y_min': 0,
  'position': 46134,
  'region_x_max': 233,
  'region_y_max': 198},
 'level': 8,
 'level_x': 0,
 'level_y': 0,
 'magnification': 20.0,
 'mm_x': 0.00044152059693584704,
 'mm_y': 0.00044152059693584704,
 'tile': array([[[254, 253, 248, 255],
         [254, 253, 248, 255],
         [254, 253, 248, 255],
         ...,
         [254, 253, 248, 255],
         [254, 253, 248, 255],
         [254, 253, 248, 255]],
 
        [[254, 253, 248, 255],
         [254, 253, 248, 255],
         [254, 253, 248, 255],
         ...,
         [254, 253, 248, 255],
         [254, 253, 248, 255],
         [254, 253, 248, 255]],
 
        [[254, 253, 248, 255],
         [254, 253, 248, 255],
         [254, 253, 248, 255],
         ...,
         [254, 253, 248, 255],
         [254, 253, 248, 255],
         [254, 25

In [152]:
num_tiles = 0

tile_means = []
tile_areas = []

mag = ts.getMetadata()['magnification']

tensor = []
predictionBatch = 128 #Different from the actual batch used in the net
BATCH_SIZE = 32

for tile_info in ts.tileIterator(
    region=dict(left=0, top=0, width=ts.sizeX, height=ts.sizeY, units='base_pixels'),
    scale=dict(magnification=mag),
    tile_size=dict(width=224, height=224),
    format=large_image.tilesource.TILE_FORMAT_NUMPY
):

    tile = tf.convert_to_tensor(tile_info['tile'][:,:,:3],dtype=tf.float32)
    tensor.append([tile])
    #print(tensor)

    if (num_tiles+1)%predictionBatch==0:
      tensor = tf.concat(tensor,axis=0)

      dataset = tf.data.Dataset.from_tensor_slices(tensor)
      dataset = dataset.batch(BATCH_SIZE)

      prediction = vgg_model.predict(dataset)
      print(prediction)

      tensor = []
      break    

    '''
    if num_tiles == 100:
        print('Tile-{} = '.format(num_tiles))
        display(tile_info)

    im_tile = np.array(tile_info['tile'])
    tile_mean_rgb = np.mean(im_tile[:, :, :3], axis=(0, 1))

    tile_means.append( tile_mean_rgb )
    tile_areas.append( tile_info['width'] * tile_info['height'] )'''

    num_tiles += 1


[[7.54333474e-03 2.36321321e-05 9.92433071e-01]
 [8.04510247e-03 2.30981350e-05 9.91931856e-01]
 [1.15241539e-02 3.32130367e-05 9.88442600e-01]
 [7.97408540e-03 2.23420011e-05 9.92003500e-01]
 [7.67957838e-03 1.87699843e-05 9.92301702e-01]
 [1.16358781e-02 2.68396852e-05 9.88337338e-01]
 [1.04017304e-02 3.16470323e-05 9.89566684e-01]
 [7.02700345e-03 2.28122735e-05 9.92950201e-01]
 [6.79352088e-03 2.27584824e-05 9.93183672e-01]
 [6.97556324e-03 2.09317295e-05 9.93003547e-01]
 [8.78374837e-03 2.73156711e-05 9.91188884e-01]
 [6.71947561e-03 1.78122409e-05 9.93262708e-01]
 [6.98789582e-03 1.79274011e-05 9.92994130e-01]
 [5.91078261e-03 1.14067198e-05 9.94077802e-01]
 [9.80721973e-03 3.21124135e-05 9.90160704e-01]
 [1.08412523e-02 2.91715987e-05 9.89129543e-01]
 [9.96783841e-03 3.05161448e-05 9.90001678e-01]
 [1.22717256e-02 3.72096256e-05 9.87691045e-01]
 [1.05253346e-02 2.74610247e-05 9.89447236e-01]
 [8.69282521e-03 2.16028129e-05 9.91285503e-01]
 [6.34008087e-03 1.80344014e-05 9.936419

In [154]:
prediction.sum(axis=0)

array([1.0539988e+00, 2.8739192e-03, 1.2694311e+02], dtype=float32)

In [157]:
x_train[0]

'/content/drive/MyDrive/meso_san_luigi/M-10.ndpi'